# Simulacrum demo

*First class syntax support for type classes in Scala *

https://github.com/mpilquist/simulacrum

Let's not only load a standard dependency, but also add a **compiler plugin**. Ignore the annoying output from the Ivy resolution.

In [1]:
load.compiler.ivy("org.scalamacros" % "paradise_2.11.6" % "2.1.0-M5")
load.ivy("com.github.mpilquist" %% "simulacrum" % "0.3.0")

:: problems summary ::
:::: WARNINGS
	Unable to reparse com.github.alexarchambault.jupyter#jupyter-scala-api_2.11.6;0.2.0-SNAPSHOT from sonatype-snapshots, using Wed May 20 00:01:36 CEST 2015

	Sorting results from com.github.alexarchambault.jupyter#jupyter-scala-api_2.11.6;0.2.0-SNAPSHOT, using Wed May 20 00:01:36 CEST 2015 and Wed May 20 00:01:36 CEST 2015

	Sorting results from com.github.alexarchambault.jupyter#jupyter-scala-api_2.11.6;0.2.0-SNAPSHOT, using Wed May 20 00:01:36 CEST 2015 and Wed May 20 00:01:36 CEST 2015

	Choosing sonatype-snapshots for com.github.alexarchambault.jupyter#jupyter-scala-api_2.11.6;0.2.0-SNAPSHOT

	Unable to reparse com.github.alexarchambault#ammonite-api_2.11.6;0.3.1-SNAPSHOT from sonatype-snapshots, using Wed May 20 00:00:10 CEST 2015

	Sorting results from com.github.alexarchambault#ammonite-api_2.11.6;0.3.1-SNAPSHOT, using Wed May 20 00:00:10 CEST 2015 and Wed May 20 00:00:10 CEST 2015

	Sorting results from com.github.alexarchambault#ammonite-api

## Examples from simulacrum README
We'll review *simulacrum* examples, and programatically inspect their generated code along the way.

First, let's import the content of the `simulacrum` package, and define a type class.

In [2]:
import simulacrum._

@typeclass trait Semigroup[A] {
  @op("|+|") def append(x: A, y: A): A
}

import simulacrum._
defined trait Semigroup

Everything seems fine... but what did the `@typeclass` annotation macro do?
Let's look at that, using the following tools from scala-reflect:

In [3]:
import scala.reflect.runtime.universe.{ reify, showCode }

import scala.reflect.runtime.universe.{ reify, showCode }

In [4]:
show(showCode(reify {
  @typeclass trait Semigroup[A] {
    @op("|+|") def append(x: A, y: A): A
  }
}.tree))

res3: ammonite.pprint.Show[String] = """
{
  trait Semigroup[A] extends AnyRef {
    def append(x: A, y: A): A
  };
  object Semigroup extends AnyRef {
    def apply[A](implicit instance: Semigroup[A]): Semigroup[A] = instance;
    trait Ops[A] extends AnyRef {
      def typeClassInstance: Semigroup[A];
      def self: A;
      def |+|(y: A): A = Ops.this.typeClassInstance.append(Ops.this.self, y)
    };
    trait ToSemigroupOps extends AnyRef {
      implicit def toSemigroupOps[A](target: A)(implicit tc: Semigroup[A]): Semigroup.this.Ops[A] = {
        final class $anon extends Semigroup.this.Ops[A] {
          val self = target;
          val typeClassInstance = tc
        };
        new $anon()
      }
    };
    trait AllOps[A] extends Semigroup.this.Ops[A] {
      def typeClassInstance: Semigroup[A]
    };
    object ops extends AnyRef {
      implicit def toAllSemigroupOps[A](target: A)(implicit tc: Semigroup[A]): Semigroup.this.AllOps[A] = {
        final class $anon extends Sem

A bit rough, but still read-able.

Now let's use that: we'll define a `Semigroup` instance for `Int`, and use it straightaway.

In [5]:
implicit val semigroupInt: Semigroup[Int] = new Semigroup[Int] {
  def append(x: Int, y: Int) = x + y
}

import Semigroup.ops._
1 |+| 2 // 3

semigroupInt: cmd4.INSTANCE.$ref$cmd1.Semigroup[Int] = cmd4$$user$$anon$1@596468fe
import Semigroup.ops._
res4_2: Int = 3

It just works :-)

Now a type class inheriting another type class, from the simulacrum README too.

In [6]:
@typeclass trait Monoid[A] extends Semigroup[A] {
  def id: A
}

defined trait Monoid

And what the `@typeclass` macro actually generated:

In [7]:
show(showCode(reify {
  @typeclass trait Monoid[A] extends Semigroup[A] {
    def id: A
  }
} .tree))

res6: ammonite.pprint.Show[String] = """
{
  trait Monoid[A] extends cmd6.$ref$cmd1.Semigroup[A] {
    def id: A
  };
  object Monoid extends AnyRef {
    def apply[A](implicit instance: Monoid[A]): Monoid[A] = instance;
    trait Ops[A] extends AnyRef {
      def typeClassInstance: Monoid[A];
      def self: A
    };
    trait ToMonoidOps extends AnyRef {
      implicit def toMonoidOps[A](target: A)(implicit tc: Monoid[A]): Monoid.this.Ops[A] = {
        final class $anon extends Monoid.this.Ops[A] {
          val self = target;
          val typeClassInstance = tc
        };
        new $anon()
      }
    };
    trait AllOps[A] extends Monoid.this.Ops[A] with cmd6.$ref$cmd1.Semigroup.AllOps[A] {
      def typeClassInstance: Monoid[A]
    };
    object ops extends AnyRef {
      implicit def toAllMonoidOps[A](target: A)(implicit tc: Monoid[A]): Monoid.this.AllOps[A] = {
        final class $anon extends Monoid.this.AllOps[A] {
          val self = target;
          val typeClassInsta

The following line should make simulacrum output what it generated, but it doesn't here for an unknown reason :|

In [8]:
System.setProperty("simulacrum.trace", "")

res7: java.lang.String = null

This is what simulacrum checks ([here](https://github.com/mpilquist/simulacrum/blob/master/core/src/main/scala_2.11/simulacrum/typeclass.scala#L58)):

In [9]:
sys.props.get("simulacrum.trace").isDefined

res8: Boolean = true

And re-initializing the compiler instance, with

In [10]:
interpreter.init()

or even with the `-verbose` option,

In [11]:
interpreter.init(Seq("-verbose"))

In [12]:
// no debug output :|
@typeclass trait Semigrp[A] {
  @op("|+|") def append(x: A, y: A): A
}

defined trait Semigrp

Nope, no debug output. But we can call ourselves `showCode` and `reify`, as above. So we don't really need that.

### Now more complex examples

From simulacrum examples directory. Comments are from simulacrum. Names were changed to prevent clash with the above examples.

In [13]:
@typeclass trait Functor[F[_]] {
  def map[A, B](fa: F[A])(f: A => B): F[B]
}

@typeclass trait Applicative[F[_]] extends Functor[F] {
  def pure[A](a: => A): F[A]
  def ap[A, B](fa: F[A])(f: F[A => B]): F[B]
  override def map[A, B](fa: F[A])(f: A => B): F[B] =
    ap(fa)(pure(f))
}

defined trait Functor
defined trait Applicative

In [14]:
@typeclass trait Monad[F[_]] extends Applicative[F] {
  @op(">>=", alias = true) def flatMap[A, B](fa: F[A])(f: A => F[B]): F[B]
  override def ap[A, B](fa: F[A])(f: F[A => B]): F[B] =
    flatMap(f)(map(fa))
  override def map[A, B](fa: F[A])(f: A => B): F[B] =
    flatMap(fa)(a => pure(f(a)))
}

@typeclass trait PlusEmpty[F[_]] {
  def empty[A]: F[A]
}

@typeclass trait MonadPlus[F[_]] extends Monad[F] with PlusEmpty[F] {
  self =>
  class WithFilter[A](fa: F[A], p: A => Boolean) {
    def map[B](f: A => B): F[B] = self.map(filter(fa)(p))(f)
    def flatMap[B](f: A => F[B]): F[B] = self.flatMap(filter(fa)(p))(f)
    def withFilter(q: A => Boolean): WithFilter[A] = new WithFilter[A](fa, x => p(x) && q(x))
  }

  def withFilter[A](fa: F[A])(p: A => Boolean): WithFilter[A] = new WithFilter[A](fa, p)
  def filter[A](fa: F[A])(f: A => Boolean) =
    flatMap(fa)(a => if (f(a)) pure(a) else empty[A])
}

defined trait Monad
defined trait PlusEmpty
defined trait MonadPlus

In [15]:
sealed trait Maybe[+A]
case class Just[A](value: A) extends Maybe[A]
case object Empty extends Maybe[Nothing]

object Maybe {
    
  def just[A](a: A): Maybe[A] = Just(a)
  def empty[A]: Maybe[A] = Empty

  implicit val instance: MonadPlus[Maybe] = new MonadPlus[Maybe] {
    def pure[A](a: => A) = just(a)
    def empty[A] = Maybe.empty[A]
    def flatMap[A, B](fa: Maybe[A])(f: A => Maybe[B]) = fa match {
      case Just(a) => f(a)
      case e @ Empty => e
    }
  }
    
}

defined trait Maybe
defined class Just
defined object Empty
defined object Maybe

In [28]:
import MonadPlus.ops._

// We get the map function from Functor.Ops, which is the super-super-super class of MonadPlus.Ops
Maybe.just(1) map ((_: Int) + 1) // shouldBe Maybe.just(2)

// We get >>= syntax as an alias for flatMap from the super-class of MonadPlus.Ops
val recriprocal: Int => Maybe[Double] = x => if (x == 0) Maybe.empty else Maybe.just(1.0 / x)
Maybe.just(1) >>= recriprocal

// We get map from Functor.Ops, flatMap from Monad.Ops, and filter from MonadPlus.Ops
def div(x: Maybe[Int], y: Maybe[Int]): Maybe[Double] = for {
  xx <- x
  yy <- y
  if (yy > 0)
} yield xx.toDouble / yy

div(Maybe.just(1), Maybe.just(2)) // shouldBe Maybe.just(1.toDouble / 2)
div(Maybe.just(1), Maybe.empty) // shouldBe Maybe.empty

import MonadPlus.ops._
res26_1: cmd26.INSTANCE.$ref$cmd25.Maybe[Int] = Just(2)
recriprocal: Int => cmd26.INSTANCE.$ref$cmd25.Maybe[Double] = <function1>
res26_3: cmd26.INSTANCE.$ref$cmd25.Maybe[Double] = Just(1.0)
defined function div
res26_5: cmd26.INSTANCE.$ref$cmd25.Maybe[Double] = Just(0.5)
res26_6: cmd26.INSTANCE.$ref$cmd25.Maybe[Double] = Empty

In [16]:
@typeclass trait Equal[A] {
  @op("=#=") def equal(x: A, y: A): Boolean
}

@typeclass trait Semigroup0[A] {
  @op("||+||") def append(x: A, y: A): A
}

@typeclass trait Monoid[A] {
  def id: A
}

implicit val intInstance: Equal[Int] with Semigroup0[Int] = new Equal[Int] with Semigroup0[Int] {
  def equal(x: Int, y: Int) = x == y
  def append(x: Int, y: Int) = x + y
  def id: Int = 0
}


defined trait Equal
defined trait Semigroup0
defined trait Monoid
intInstance: $user.Equal[Int] with $user.Semigroup0[Int] = cmd15$$user$$anon$4@167637e4

In [17]:
// We cannot import Equal.Ops and Semigroup.Ops because of the name clash
// However, an alias for the implicit conversion is generated which allows direct import
{
  import Equal.ops._, Semigroup0.ops._
  (1 ||+|| 2) =#= (2 ||+|| 1)
}

// Alernatively, multiple type class ops can be combined in to a syntax object, which provides
// a single import for all implicit conversions
{
  object all extends Equal.ToEqualOps with Semigroup0.ToSemigroup0Ops with Monoid.ToMonoidOps
  import all._
  (1 ||+|| 2) =#= (2 ||+|| 1)
}

res16_0: Boolean = true
res16_1: Boolean = true